### 协议栈
* 应用层，表示层，会话层
* 传输层(TCP/UDP)
* 网络层(IP)
* 链路层
* 物理层

### I. 应用层
#### 1.1 用requests - http library for human 做http/https请求

In [59]:
import requests

proxies = {
    "http": "http://proxy.bloomberg.com:80",
    "https": "https://proxy.bloomberg.com:80",
}

def req(url):
    # in BBL we need proxy to forward the request out
    resp = requests.post(url, proxies=proxies)
    return resp, resp.status_code

In [32]:
from bs4 import BeautifulSoup
def parse_html(source):
    soup = BeautifulSoup(source)
    return soup

In [67]:
resp, status_code = req("https://music.163.com/")

raw = resp.content.decode("utf-8")
soup = parse_html(raw)

print(status_code, soup.title.string)

200 网易云音乐


#### 使用http.client做应用层的连接

In [56]:
import http.client

def http_req(address):
    
    connection = http.client.HTTPSConnection("proxy.bloomberg.com", 80)
    connection.set_tunnel(address)
    connection.request('GET', "/index.html")
    print(dir(connection))
    
    print("Socket used: ", connection.sock)
    raw_reply = connection.getresponse().read()
    
    reply = raw_reply.decode("utf-8")
    return reply

In [57]:
raw = http_req("www.python.org")

soup = parse_html(raw)
print(soup.title.string)

['_HTTPConnection__response', '_HTTPConnection__state', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_buffer', '_check_hostname', '_context', '_create_connection', '_get_hostport', '_http_vsn', '_http_vsn_str', '_method', '_output', '_send_output', '_send_request', '_set_content_length', '_tunnel', '_tunnel_headers', '_tunnel_host', '_tunnel_port', 'auto_open', 'cert_file', 'close', 'connect', 'debuglevel', 'default_port', 'endheaders', 'getresponse', 'host', 'key_file', 'port', 'putheader', 'putrequest', 'request', 'response_class', 'send', 'set_debuglevel', 'set_tunnel', 'sock', 'source_address', 'timeout']
<ssl.SSLSocket fd=1592, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('10.1

#### 用套接字做更底层链接

In [95]:
import socket


def socket_req(request_text):
    sock = socket.socket()
    sock.connect(("python.org", 80))
    print(dir(sock))
    sock.sendall(request_text.encode("ascii"))
    
    raw_reply = b''
    while True:
        more = sock.recv(4096)
        if not more:
            break
        raw_reply += more
    print(raw_reply.decode("utf-8"))
    sock.close()

In [100]:
request_text = '''
GET / HTTP/1.1\r\n\
Host: python.org\r\n\
Connection: KEEP-ALIVE\r\n\
\r\n\
'''
socket_req(request_text)

['__class__', '__delattr__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_accept', '_check_sendfile_params', '_closed', '_decref_socketios', '_io_refs', '_real_close', '_sendfile_use_send', '_sendfile_use_sendfile', 'accept', 'bind', 'close', 'connect', 'connect_ex', 'detach', 'dup', 'family', 'fileno', 'get_inheritable', 'getpeername', 'getsockname', 'getsockopt', 'gettimeout', 'ioctl', 'listen', 'makefile', 'proto', 'recv', 'recv_into', 'recvfrom', 'recvfrom_into', 'send', 'sendall', 'sendfile', 'sendto', 'set_inheritable', 'setblocking', 'setsockopt', 'settimeout', 'share', 'shutdown', 'timeout', 'type']



* compared with requests

In [99]:
resq, status_code = req("https://python.org")
resq.request.method, resq.request.headers, resq.request.url, resq.request.path_url

('GET',
 {'User-Agent': 'python-requests/2.22.0', 'Accept-Encoding': 'gzip, deflate', 'Connection': 'keep-alive', 'Accept': '*/*'},
 'https://www.python.org/',
 '/')

### II. 传输层
#### 2.1 UDP
IP协议只负责尝试把每个数据包传输到正确的机器, 如果两个应用程序需要维护一个会话，需要两个额外的特性，由IP层以上的协议提供
* 给数据包打标签，把数据包和链路上其他与该机器进行的会话区分开，-> multiplexing 多路复用
* 数据包有错误，发送方需要进行修复，接受发需要复原顺序错乱的数据包，放弃重复的数据包。-> reliable transport


* TCP 解决上述两个问题，UDP 只解决第一个

In [109]:
socket.getservbyname("domain")

53

##### 2.1.1 Socket

##### 2.1.2 UDP sample

In [1]:
from datetime import datetime
import socket
MAX_BYTES = 65535

def server(port):
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    sock.bind(('127.0.0.1', port))
    
    print("Server:::Socket fileno {} \n".format(sock.fileno()))
    print("Server:::Listening at {}\n".format(sock.getsockname()))
    
    while True:
        data, address = sock.recvfrom(MAX_BYTES)
        text = data.decode('ascii')
        
        print("Server:::The client {} says {}\n".format(address, text))
        

def client(port):
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    data = "The time is {}\n".format(datetime.now()).encode("ascii")
    
    sock.sendto(data, ("127.0.0.1", port))
    print("Client:::The OS assigned me the address {}\n".format(sock.getsockname()))

In [2]:
from concurrent.futures import ThreadPoolExecutor
executor = ThreadPoolExecutor(max_workers=2)

a = executor.submit(server, 9233)

Server:::Socket fileno 1204 

Server:::Listening at ('127.0.0.1', 9233)



In [3]:
b = executor.submit(client, 9233)

Server:::The client ('127.0.0.1', 61045) says The time is 2019-06-11 09:45:58.852302

Client:::The OS assigned me the address ('0.0.0.0', 61045)




* concurrent.futures 里的thread可以通过内存queue交互么，还是必须通过socket ？？？

###### Socket vs Fileno

In [9]:
f = open("Design Pattern.ipynb")
print("File descriptor: ", f.fileno())
f.close()

File descriptor:  4


##### 2.1.3 UDP broadcast

In [1]:
import socket
from concurrent.futures import ThreadPoolExecutor

def broadcast_server(intrface, port):
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    sock.bind((interface, port))
    
    print("BroadcastServer:::Listening for datagrams at {}".format(sock.getsockname()))
    
    while True:
        data, address = sock.recvfrom(65535)
        print("BroadcastServer:::The client {} says {}\n".format(address, data.decode("ascii")))

def client(network, port):
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    sock.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, 1)
    data = "Broadcast datagram!".encode("ascii")
    sock.sendto(data, (network, port))

In [2]:
b_executor = ThreadPoolExecutor(max_workers=4)
b_executor.submit(broadcast_server, "0.0.0.0", 9123)   # ?? why cannot connect to 0.0.0.0

<Future at 0x1d9fabbc8d0 state=finished raised NameError>

In [3]:
b_executor.submit(client, "0.0.0.0", 9123)

<Future at 0x1d9fabbf3c8 state=finished raised OSError>

#### 2.2 TCP

In [8]:
import socket

def recvall(sock, length):
    data = b''
    while len(data) < length:
        more = sock.recv(length - len(data))
        
        if not more:
            raise Exception("Lost pacakge")
        
        data += more
    return data


def tcp_server(interface, port):
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    sock.bind((interface, port))
    
    # listen 可以传入一个int 参数，表示等待连接的最大数目。如果服务器在处理请求时有新的握手请求，则该请求会被压栈
    # int 表示栈的空间，如果超过，系统忽略新请求
    sock.listen(1)
    
    print("Listening at ", sock.getsockname())
    
    while True:
        # accpet() connect() 代表了 TCP 建立连接握手的过程，与 UDP 很大不同
        # accept 是服务器程序监听端口，等待客户端发起握手请求，建立会话
        sc, sockname = sock.accept()
        print("We have accepted a connection from {} \n".format(sockname))
        print(" Socket Name: {} \n".format(sc.getsockname()))
        print(" Socket Peer: {} \n".format(sc.getpeername()))
        
        message = recvall(sc, 16)
        print(" Incoming sixteen-octet message: {}\n".format(message))
        
        sc.sendall(b"Farewell, client")
        sc.close()

        
def client(host, port):
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    # TCP connect
    # client 发起握手请求，与客户端进行三次握手，成功则 connect 成功， 可以发送数据
    sock.connect((host, port))   
    print("Client has assigned socket name {}\n".format(sock.getsockname()))
    
    sock.sendall(b"Hi there, ")
    sock.sendall(b"server")    # client 分两次发送，server还是等16字节接收
    reply = recvall(sock, 16)
    print("The server said {} \n".format(reply))
    sock.close()

In [9]:
from concurrent.futures import ThreadPoolExecutor
tcp_executor = ThreadPoolExecutor(max_workers=2)
tcp_executor.submit(tcp_server, "127.0.0.1", 9123)   

<Future at 0x231f3c72978 state=running>

Listening at  ('127.0.0.1', 9123)


In [10]:
tcp_executor.submit(client, "127.0.0.1", 9123)   

<Future at 0x231f3c726a0 state=running>

Client has assigned socket name ('127.0.0.1', 51974)
We have accepted a connection from ('127.0.0.1', 51974) 


The server said b'Farewell, client' 
 Socket Name: ('127.0.0.1', 9123) 


 Socket Peer: ('127.0.0.1', 51974) 

 Incoming sixteen-octet message: b'Hi there, server'



### III. 网络层
#### 3. 1 IP 地址
IP协议只负责尝试把每个数据包传输到正确的机器
* IPV4
* IPV6

* DNS, Domain Name System

In [108]:
socket.gethostbyname("www.python.org")

'10.124.7.14'

###### 3.2 路由
* 

### VII 服务器架构
#### 7.1 

* 保证DNS 发布的ip地址一致有效的实用方案： 负载均衡 + 反向代理
* PaaS： Platform as a Service

#### 7.2 构建一个单线程的server
* server是单线程的

In [4]:
import socket, time

def create_srv_socket(address):
    
    listener = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    listener.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)  ## ???
    
    listener.bind(address)
    
    listener.listen(64)
    print("Server::: Listen at {}\n".format(address))
    
    return listener


def accept_connections_forever(listener):
    
    while True:
        sock, address = listener.accept()
        print("Accept connection from {}\n".format(address))
        handle_conversation(sock, address)

        
def handle_conversation(sock, address):
    try:
        while True:
            handle_reuqest(sock)
    except EOFError:
        print("Client socket to {} has closed\n".format(address))
    except Exception as e:
        print("Client {} error: {}\n".format(address, e))
    finally:
        sock.close()

        
def get_anwser(question):
    time.sleep(1.0)
    return question.replace(b'?', b'.')


def handle_reuqest(sock):
    question = recv_until(sock, b'?')
    answer = get_anwser(question)
    sock.sendall(answer)

    
def recv_until(sock, suffix):
    message = sock.recv(4096)
    
    if not message:
        raise EOFError("Socket closed\n")
    while not message.endswith(suffix):
        data = sock.recv(4096)
        if not data:
            raise IOError("received {!r} then socket closed\n".format(message))
        message += data
    return message

##### 7.3 单进程服务器

In [2]:
def timeit(func):
    
    def wrapper(*args, **kwargs):
        start = time.time()
        resp = func(*args, **kwargs)
        print("Funcition running for {}\n".format(time.time() - start))
        return resp
    return wrapper


def single_thread_server(address):
    listener = create_srv_socket(address)
    accept_connections_forever(listener)

@timeit
def client(address, questions, cause_error=False):
    
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.connect(address)
    
    
    if cause_error:
        sock.sendall(question[:-1])
        return
    
    for q in questions:
        sock.sendall(q)
        print("Client recieved {}\n".format(recv_until(sock, b'.')))
    
    sock.close()    

In [3]:
from concurrent.futures import ThreadPoolExecutor


def make_requests(arch_executor, port):
    arch_executor.submit(client, ("127.0.0.1", port), [b"Yoyo?"])
    arch_executor.submit(client, ("127.0.0.1", port), [b"Puyo?"])
    arch_executor.submit(client, ("127.0.0.1", port), [b"HIHI?"])
    

def run(port, server):
    arch_executor = ThreadPoolExecutor(max_workers=5)
    arch_executor.submit(server, ("127.0.0.1", port))
    make_requests(arch_executor, port)

In [4]:
run(9128, single_thread_server)

Server::: Listen at ('127.0.0.1', 9128)

Accept connection from ('127.0.0.1', 56833)

Client recieved b'Yoyo.'

Funcition running for 1.015542984008789
Client socket to ('127.0.0.1', 56833) has closed


Accept connection from ('127.0.0.1', 56836)

Client recieved b'Puyo.'

Client socket to ('127.0.0.1', 56836) has closed
Funcition running for 2.0163705348968506


Accept connection from ('127.0.0.1', 56837)

Client recieved b'HIHI.'

Client socket to ('127.0.0.1', 56837) has closed
Funcition running for 3.023984909057617




##### 7.4 多进程服务器
* 这个实现只是主线程sprawn了多个线程用于监听
    * 线程管理呢
    * 如果有线程崩溃呢
    * Solution，主线程可以作为监控线程，来检查n个服务器线程的运行情况
* 用哪个多线程库
    * threading.Thread 线程之间共享内存
    * multiprocessing.Process 操作系统会为线程分配独立的内存空间和文件描述付
        * 会增加系统开销
        * 但是线程被隔离，降低服务器线程导致主监控进程崩溃的情况
        

In [5]:
from threading import Thread

def create_threadpool(listener, workers=4):
    t = (listener, )
    try:
        for i in range(workers):
            thread = Thread(target=accept_connections_forever, args=t)
            thread.start()
            print("Create thread {} for listen client request\n".format(thread.name))
    except Exception as e:
        print("Error when sprawn threads {}\n".format(e))
            
        
def multi_thread_server(address):
    listener = create_srv_socket(address)
    create_threadpool(listener)

In [6]:
run(9129, multi_thread_server)

Server::: Listen at ('127.0.0.1', 9129)

Create thread Thread-14 for listen client request

Create thread Thread-15 for listen client request

Create thread Thread-16 for listen client request

Accept connection from ('127.0.0.1', 56855)
Accept connection from ('127.0.0.1', 56854)
Accept connection from ('127.0.0.1', 56856)
Create thread Thread-17 for listen client request




Client recieved b'HIHI.'
Client socket to ('127.0.0.1', 56856) has closed
Client recieved b'Yoyo.'
Client socket to ('127.0.0.1', 56854) has closed
Client recieved b'Puyo.'
Client socket to ('127.0.0.1', 56855) has closed






Funcition running for 1.038597822189331
Funcition running for 1.0465691089630127
Funcition running for 1.0535643100738525





#### 7.4 异步服务器
* 到目前为止我们一直使用阻塞socket
* 为了实现异步，更好的利用CPU，减少在IO中等待的时间
    * 使用非阻塞socket

##### 7.4.1 非阻塞socket
* 非阻塞socket在进行send()或者recv()调用时永远不会阻塞调用进程
* 异步表示，服务器代码从来不会停下来等待某个客户端
* 操作系统提供很多调用来支持异步
    * 最古老的是POSIX的select()调用
    * select的替代品，Linux的poll()
    * BSD系统的epoll()

##### 7.4.2 用select实现简单的异步

``` python
import select                                                                                                                
import socket                                                                                                                
import time                                                                                                                  
                                                                                                                             
                                                                                                                             
def create_srv_socket(address):                                                                                                                                                                                                              
    listener = socket.socket(socket.AF_INET, socket.SOCK_STREAM)                                                             
    listener.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)  # ???                                                    
                                                                                                                             
    listener.bind(address)                                                                                                   
                                                                                                                             
    listener.listen(64)                                                                                                      
    print("Server::: Listen at {}\n".format(address))                                                                        
                                                                                                                             
    return listener                                                                                                          
                                                                                                                             
                                                                                                                             
class AsyncServer(object):                                                                                                   
                                                                                                                             
    def __init__(self):                                                                                                      
        self.addresses = {}                                                                                                  
        self.bytes_received = {}                                                                                             
        self.bytes_to_send = {}                                                                                              
        self.sockets = {}                                                                                                    
                                                                                                                             
    def all_events_forever(self, poll_object):                                                                               
                                                                                                                             
        while True:                                                                                                          
            for fd, event in poll_object.poll():                                                                             
                yield fd, event                                                                                              
                                                                                                                             
    def handle_events(self, listener):                                                                                       
        self.sockets[listener.fileno()] = listener                                                                           
                                                                                                                             
        poll_object = select.poll()                                                                                          
        poll_object.register(listener, select.POLLIN)                                                                        
                                                                                                                             
        for fd, event in self.all_events_forever(poll_object):                                                               
            sock = self.sockets[fd]                                                                                          
                                                                                                                             
            if event & (select.POLLHUP | select.POLLERR | select.POLLNVAL):                                                  
                if sock.fileno() != listener.fileno():                                                                       
                    self.connection_close(sock, poll_object)                                                               
                                                                                                                             
            elif sock is listener:                                                                                           
                self.accept_connection(sock, poll_object)                                                                    
                                                                                                                             
            elif event & select.POLLIN:                                                                                      
                self.poll_in(sock, poll_object)                                                                              
                                                                                                                             
            elif event & select.POLLOUT:                                                                                     
                self.poll_out(sock, poll_object)                                                                             
                                                                                                                             
    def accept_connection(self, sock, poll_object):                                                                          
        sock, address = sock.accept()                                                                                        
        print("Accept connection from {}".format(address))                                                                   
        sock.setblocking(False)                                                                                              
        self.sockets[sock.fileno()] = sock                                                                                   
        self.addresses[sock.fileno()] = address                                                                              
        poll_object.register(sock, select.POLLIN)                                                                            
                                                                                                                             
    def connection_close(self, sock, poll_object):                                                                           
        if sock.fileno() < 0:                                                                                                
            return       
        
        print("Closing ", sock)                                                                                              
        address = self.addresses.pop(sock.fileno())                                                                          
                                                                                                                             
        rb = self.bytes_received.pop(sock.fileno(), b'')                                                                     
        sb = self.bytes_to_send.pop(sock.fileno(), b'')                                                                      
                                                                                                                             
        if rb:                                                                                                               
            print("Client {} sent {} but then closed".format(address, rb))                                                   
        elif sb:                                                                                                             
            print("Client {} closed before we sent".format(address, sb))                                                     
        else:                                                                                                                
            print("Client {} closed socket normally".foramt(address))                                                        
                                                                                                                             
        poll_object.unregister(sock.fileno())                                                                                
        del self.sockets[sock.fileno()]                                                                                      
                                                                                                                                                                                                                                                      
    def get_answer(self, data):                                                                                              
        time.sleep(0.0)                                                                                                      
        return data.replace(b'?', b'.')                                                                                      
                                                                                                                             
    def poll_in(self, sock, poll_object):                                                                                    
        more_data = sock.recv(4096)                                                                                          
                                                                                                                             
        if not more_data:                                                                                                    
            sock.close()  # next poll() would be POLLNVAL, thus clean up                                                     
            return                                                                                                           
                                                                                                                             
        data = self.bytes_received.pop(sock.fileno(), b'') + more_data                                                       
                                                                                                                             
        if data.endswith(b'?'):                                                                                              
            self.bytes_to_send[sock.fileno()] = self.get_answer(data)                                                        
            poll_object.modify(sock.fileno(), select.POLLOUT)                                                                
        else:                                                                                                                
            self.bytes_received[sock.fileno()] = data                                                                        
                                                                                                                             
    def poll_out(self, sock, poll_object):                                                                                   
        data = self.bytes_to_send[sock.fileno()]                                                                             
        n = sock.send(data)                                                                                                  
                                                                                                                             
        if n < len(data):                                                                                                    
            self.bytes_to_send[sock.fileno()] = data[n:]                                                                     
        else:                                                                                                                
            poll_object.modify(sock.fileno(), select.POLLIN)                                                                 
                                                                                                                             
                                                                                                                             
if __name__ == "__main__":   
    address = ("127.0.0.1", 9382)                                                                                            
    listener = create_srv_socket(address)                                                                                    
    server = AsyncServer()      
```

In [ ]:
from select import select
from collections import deque
import traceback


def log_exception(func):
    
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            print(traceback.format_exc())
            print("Exception {}\n".format(e))
    
    return wrapper

class YieldEvent:
    def handle_yield(self, sched, task):
        pass
    
    def handle_resume(self, sched, task):
        pass

    
class Scheduler:
    
    def __init__(self):
        self._numtasks = 0
        self._ready = deque()
        self._input_queue = {}
        self._output_queue = {}
    
    def _iopoll(self):
        # select item from 3个通信列表，in, out, exception
        rset, wset, eset = select(
            self._input_queue, self._output_queue, [])
        
        for r in rset:
            evt, task = self._input_queue.pop(r)
            evt.handle_resume(self, task)
        
        for w in wset:
            evt, task = self._output_queue.pop(w)
            evt.handle_resume(self, task)
    
    def new(self, task):
        self._ready.append((task, None))
        self._numtasks += 1
    
    def add_ready(self, task, msg=None):
        self._ready.append((task, msg))
        
    def _read_wait(self, fileno, evt, task):
        self._input_queue[fileno] = (evt, task)
    
    def _write_wait(self, fileno, evt, task):
        self._output_queue[fileno] = (evt, task)
    
    @log_exception  
    def run(self):    
        while self._numtasks:
        
            if not self._ready:
                self._iopoll()
            task, msg = self._ready.popleft()
            
            try:
                r = task.send(msg)
                if isinstance(r, YieldEvent):
                    r.handle_yield(self, task)
                else:
                    raise RuntimeError('Unrecognized yield event')
            except StopIteration:
                self._numtasks -= 1
                
@log_exception  
def recieve_line(sock):
    chars = []
    while True:
        c = yield sock.recv(1)
        if not c:
            break
        
        chars.append(c)
        
        if c == b'\n':
            break
    print("Recieved data ", chars)
    return b''.join(chars)


class ReadSocket(YieldEvent):
    
    def __init__(self, sock, nbytes):
        self.sock = sock
        self.nbytes = nbytes
    
    def handle_yield(self, sched, task):
        sched._read_wait(self.sock.fileno(), self, task)
    
    def handle_resume(self, sched, task):
        data = self.sock.recv(self.nbytes)
        sched.add_ready(task, data)
    
class WriteSocket(YieldEvent):
    
    def __init__(self, sock, data):
        self.sock = sock
        self.data = data
    
    def handle_yield(self, sched, task):
        sched._write_wait(self.sock.fileno(), self, task)
    
    def handle_resume(self, sched, task):
        nsent = self.sock.send(self.data)
        sched.add_ready(task, nsent)
    
class AcceptSocket(YieldEvent):
    
    def __init__(self, sock):
        self.sock = sock
    
    def handle_yield(self, sched, task):
        sched._read_wait(self.sock.fileno(), self, task)
    
    def handle_resume(self, sched, task):
        r = self.sock.accept()
        sched.add_ready(task, r)


class Socket(object):
    
    def __init__(self, sock):
        self._sock = sock
    
    def recv(self, nbytes):
        return ReadSocket(self._sock, nbytes)
    
    def send(self, data):
        return WriteSocket(self._sock, data)
    
    def accept(self):
        return AcceptSocket(self._sock)
    
    def __getattr__(self, name):
        return getattr(self._sock, name)

In [ ]:
import socket
import time
from datetime import datetime

            
class EchoServer:
    
    def __init__(self, addr, sched):
        self.sched = sched
        self.sched.new(self.server_loop(addr))
    
    @log_exception
    def server_loop(self, addr):
        s = Socket(socket.socket(socket.AF_INET, socket.SOCK_STREAM))
        
        s.bind(addr)
        s.listen(5)
        
        while True:
            sc, socketname = yield s.accept()
            
            # 收到客户request，交给处理的协程
            print("Got one client request from {}\n".format(socketname))
            self.sched.new(self.client_handler(Socket(sc)))
            
    @log_exception
    def client_handler(self, client):
        while True:
            line = yield from recieve_line(client)
            if not line:
                break
            
            line = b'Got: ' + line
            print(line)
            yield client.send(line)
            
            client.close()
            print('Client closed')


@log_exception  
def client(host, port):
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    # TCP connect
    sock.connect((host, port))   
    print("Client has assigned socket name {}\n".format(sock.getsockname()))
    
    sock.sendall(b"Hi there, \n")
    reply = recieve_line(sock)
    print("The server said {} \n".format(reply))
    sock.close()

In [ ]:
from concurrent.futures import ThreadPoolExecutor
excutor = ThreadPoolExecutor(max_workers=2)

sched = Scheduler()
EchoServer(('127.0.0.1', 5600), sched)
excutor.submit(sched.run)

excutor.submit(client, "127.0.0.1", 5600)

* asyncio

In [1]:
import asyncio

class AsyncioServer(asyncio.Protocol):
    
    def connection_made(self, transport):
        self.transport = transport
        self.address = transport.get_extra_info('peername')
        self.data = b''
        print("Accept connection from {}".format(self.address))
    
    def data_received(self, data):
        self.data += data
        if self.data.endswith(b'?'):
            anwser = self.data.replace(b'?', b'.')
            print("Answer ", anwser)
            self.transport.write(anwser)
            self.data = b''
    
    def connection_lost(self, exc):
        if exc:
            print("Client {} error: {}".format(self.address, exc))
        elif self.data:
            print("Client {} sent {} but then closed".format(self.address, self.data))
        else:
            print("Client {} closed socket".format(self.address))

In [1]:
def serve(address, server):
    loop = asyncio.get_event_loop()
    coro = loop.create_server(server, *address)
    server = loop.run_until_complete(coro)
    print("Listening at {}".format(address))
    try:
        loop.run_forever()
    finally:
        server.colse()
        loop.close()

In [ ]:
address = ("127.0.0.1", 9823)
serve(address, AsyncioServer)

Listening at ('127.0.0.1', 9823)
Accept connection from ('127.0.0.1', 65280)
Answer  b'yoyo.'
Client ('127.0.0.1', 65280) closed socket
Accept connection from ('127.0.0.1', 65281)
Answer  b'hihi.'
Client ('127.0.0.1', 65281) closed socket
Accept connection from ('127.0.0.1', 65282)
Answer  b'heyoha.'
Client ('127.0.0.1', 65282) closed socket


```shell
Client recieved b'yoyo.'
Funcition running for 0.002490997314453125
Client recieved b'hihi.'
Funcition running for 0.0019991397857666016
Client recieved b'heyoha.'
Funcition running for 0.0020008087158203125
```

In [ ]:
import asyncio

@asyncio.coroutine
def handle_conversation(reader, writer):
    try:
        address = writer.get_extra_info("peername")

        print("Accepted connection from {}".format(address))

        while True:
            data = b''
            while not data.endswith(b'?'):
                more_data = yield from reader.read(4096)
                if not more_data:
                    if data:
                        print("Client {} sent {!r} but then closed\n".format(address, data))
                    else:
                        print("Client {} closed socket normally\n".format(address))
                    return
                data += more_data

            anwser = data.replace(b'?', b'.')
            writer.write(anwser)
    except Exception as e:
        print("Exception ", e)

In [ ]:
address = ("127.0.0.1", 9823)
serve(address, handle_conversation)